Processing real estate data to determine whether to
invest money in a condo or in an ETF.

In [11]:
# We have a csv file that contains fake home sale data. We need to geocode it
# to convert addresses to X, Y coordinates. The addresses are for rental
# apartments for the St Clair area and the sales data are for actual condos
# from House Sigma (addresses not used).

# First, we need to import the needed packages.

import geopandas as gpd
import pandas as pd
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona
import warnings
warnings.filterwarnings('ignore')

from geopy.extra.rate_limiter import RateLimiter

from geopy.geocoders import Nominatim

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
from plotly.offline import plot
import matplotlib.pyplot as plt

In [2]:
# Connect to the Google Drive to get stored data

%%capture
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/

In [9]:
# Next, we need to write a function to geocode (conver to X, Y) the Toronto
# condo addresses.

def find_loc(file_name):

    #Test the geocoder using a known address
    locator = Nominatim(user_agent="myGeocoder")
    geocode = RateLimiter(locator.geocode, min_delay_seconds=0.5)

    #Get Toronto address
    lat = []
    lon = []
    #Read in the condo sales information
    add_csv = pd.read_csv('/content/drive/MyDrive/'+file_name+'.csv',sep=',')
    print(len(add_csv))
    add_csv = add_csv[~add_csv['Address'].isna()]
    count = 1
    #Loop through the addresses
    for ad in list(add_csv['Address']):
        location = locator.geocode(str(ad)+", Toronto, Ontario", timeout=None)
        print(str(ad)+", Toronto, Ontario")
        try:
            lat.append(location.latitude)
            lon.append(location.longitude)
            print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
        except (AttributeError,TimeoutError): #In case the geocoder can't find address
            lat.append('NA')
            lon.append('NA')
        count += 1

    add_csv['latitude'] = lat
    add_csv['longitude'] = lon
    return add_csv

file_name = 'fake_sales_data'

condoXY = find_loc(file_name)
print(condoXY)

80
77 St. Clair Ave E, Toronto, Ontario
Latitude = 43.6887584, Longitude = -79.3905022
77 St. Clair Ave E, Toronto, Ontario
Latitude = 43.6887584, Longitude = -79.3905022
77 St. Clair Ave E, Toronto, Ontario
Latitude = 43.6887584, Longitude = -79.3905022
40 Pleasant Blvd, Toronto, Ontario
Latitude = 43.687583599999996, Longitude = -79.39240573922037
40 Pleasant Blvd, Toronto, Ontario
Latitude = 43.687583599999996, Longitude = -79.39240573922037
40 Pleasant Blvd, Toronto, Ontario
Latitude = 43.687583599999996, Longitude = -79.39240573922037
42 Glen Elm Ave, Toronto, Ontario
Latitude = 43.6920482, Longitude = -79.39300881062545
64 St Clair Ave West, Toronto, Ontario
Latitude = 43.6878516, Longitude = -79.39719984344795
70 Delisle Ave, Toronto, Ontario
Latitude = 43.68879615, Longitude = -79.39775136760113
70 Delisle Ave, Toronto, Ontario
Latitude = 43.68879615, Longitude = -79.39775136760113
70 Delisle Ave, Toronto, Ontario
Latitude = 43.68879615, Longitude = -79.39775136760113
50 Lawton

In [10]:
 # Now, we need to calculate the cost of buying a condo and the monthly cost
 # over a period of X years

# Account for inflation
pd.set_option('display.float_format', lambda x: '%.3f' % x)
inflation_years = list(range(2000,2023))
inflation_values = [64.61,59.27,57.36,53.37,49.57,\
                        47.05,43.46,40.48,36.22,36.58,\
                        35.28,31.22,29.28,27.8,24.86,\
                        23.58,21.77,20.55,17.66,15.33,\
                        14.58,11.17,2.81]

bcsv = condoXY[condoXY['Last Sold '].notna()]


In [30]:
# Make a function to predict the 2003 selling price (if it were sold in that year)
# Using a random forest machine learning model

def get_potential_app(bdata):
    bdata = bdata.dropna(how='any')
    bdata = bdata[bdata['latitude'] != 'NA']
    bdata = bdata[bdata['longitude'] != 'NA']

    bdata_train = bdata[bdata['Last Sold '].notna()]
    for val in [['Cost','Maintenance','Bedrooms','Sq Foot','Last Sold ',\
                               'Price Last Sold','latitude','longitude']]:
      bdata_train = bdata[bdata[val].notna()]

    bdata_train = bdata_train[['Cost','Maintenance','Bedrooms','Sq Foot','Last Sold ',\
                               'Price Last Sold','latitude','longitude']]


    reg = RandomForestRegressor(
        n_estimators=100, max_features='sqrt', random_state=1)
    X_pd = bdata_train[['Cost','Maintenance','Bedrooms','Sq Foot','Last Sold ',\
                               'latitude','longitude']]
    X_train = np.array(bdata_train[['Cost','Maintenance','Bedrooms','Sq Foot','Last Sold ',\
                               'latitude','longitude']])
    y = np.array(bdata_train['Price Last Sold']).reshape(-1, 1)
    bdata['Est_2003'] = [2003]*len(bdata)
    X_test = np.array(bdata[['Cost','Maintenance','Bedrooms','Sq Foot','Est_2003',\
                               'latitude','longitude']])

    X_test_check = np.array(bdata_train[['Cost','Maintenance','Bedrooms','Sq Foot','Last Sold ',\
                               'latitude','longitude']])

    freg = reg.fit(X_train, y)
    Zi = freg.predict(X_test)
    Zc = freg.predict(X_test_check)
    bdata['If_Sold_Last_2003_Price'] = [round(x) for x in list(Zi)]

    y_true = y
    y_pred = Zc

    print('PERFORMANCE OF RANDOM FOREST MODEL..................................')
    print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_true, y_pred))
    print('Median Absolute Error:', metrics.median_absolute_error(y_true, y_pred))
    print('R^2:', metrics.r2_score(y_true, y_pred))

    #from sklearn.inspection import PartialDependenceDisplay

    #PartialDependenceDisplay.from_estimator(freg,X_pd,['Cost','Bedrooms',\
                               #'latitude','longitude'],kind='both')
    #plt.show()
    print(bdata[bdata['Last Sold '] == 2003][['Price Last Sold','If_Sold_Last_2003_Price']])
    return bdata

# Function to calculate inflation
def calc_increase(m,years=20,proptax=False):
    m_tracker = [m]
    for c in list(range(0,years)):
        m2 = m_tracker[-1] + (m_tracker[-1]*0.015)
        m_tracker.append(m2)
    if proptax == True:
        byear = m_tracker
    else:
        byear = [x*12 for x in m_tracker]
    return sum(byear)

# Make a function to calculate costs of purchasing condo & appreciation

def calc_buy_costs(price,maint,ptax,last_sold1,last_sold_price,\
                   inf_years=list(range(2000,2023)),inf_val=[64.61,59.27,57.36,53.37,49.57,\
                        47.05,43.46,40.48,36.22,36.58,\
                        35.28,31.22,29.28,27.8,24.86,\
                        23.58,21.77,20.55,17.66,15.33,\
                        14.58,11.17,2.81],\
                   years=20,percent_down=0.20,mortgage=0.05,yloan=25):
    # Calculate mortgage payment

    interest = 0.05 / 12
    loan = price * (1 - percent_down)
    total_number_of_payments = yloan * 12
    number_of_payments_paid = years
    mortgageP = loan * (interest * (1 + interest)**total_number_of_payments) \
                       / ((1 + interest)**total_number_of_payments - 1)
    remaining = loan * ((1 + interest)**total_number_of_payments - (1 + interest)**number_of_payments_paid) \
                        / ((1 + interest)**total_number_of_payments - 1)

    total= mortgageP


    # Calculate Appreciation (minus)

    ind = inf_years.index(last_sold1)
    inflation = inf_val[ind]
    before_price = last_sold_price #+ last_sold_price*(inflation/100)
    app = (price-before_price) #/ (2023-2013)
    #app = app * years


    # Maint Fees

    maint_calc = calc_increase(float(maint),years=20)

    costs_maint = mortgageP + maint_calc


    # Property Tax

    ptax_calc = calc_increase(float(ptax),years=20,proptax=True)

    costs_ptax = costs_maint + ptax_calc

    cost_adj = -costs_ptax + price + app - remaining

    agent_fees = price * 0.05

    cost_adj = cost_adj+(-agent_fees)
    down_pay = price*percent_down

    return round(cost_adj,1),round(app,1),round(down_pay,1),round(mortgageP + float(maint),1)

bcsv['total_buy'],bcsv['appreciation'],bcsv['down_pay'],bcsv['mpay'] = np.vectorize(calc_buy_costs)(bcsv['Cost'],bcsv['Maintenance'],bcsv['Property Tax'],\
                                                     bcsv['Last Sold '],bcsv['Price Last Sold'])
bcsv1 = bcsv.dropna(how='any')
bcsv1 = bcsv1[bcsv1['latitude'].notna()]
bcsv2 = get_potential_app(bcsv1)

bcsv2['total_buy2'],bcsv2['appreciation'],bcsv2['down_pay'],bcsv2['mpay'] = np.vectorize(calc_buy_costs)(bcsv2['Cost'],bcsv2['Maintenance'],bcsv2['Property Tax'],\
                                                     bcsv2['Est_2003'],bcsv2['If_Sold_Last_2003_Price'])
print('OUTPUT DATA.............................................................')
print(bcsv2)

print('AVERAGE CONDO COST TO BUY OVER 20 YEARS (+) = APPRECIATED, (-) = SPENT MONEY')
average_condo_cost = np.nanmean(bcsv2['total_buy2'])
print(average_condo_cost)

PERFORMANCE OF RANDOM FOREST MODEL..................................
Mean Absolute Error (MAE): 66590.03376623377
Median Absolute Error: 41205.0
R^2: 0.9760685024351765
    Price Last Sold  If_Sold_Last_2003_Price
11       202501.000                   236451
52       107000.000                   177450
OUTPUT DATA.............................................................
   Number             Address  Maintenance        Cost  Property Tax  \
0     336  77 St. Clair Ave E     2230.000 1365000.000      5710.000   
1     338  77 St. Clair Ave E     2303.000 2100000.000      5984.000   
2     324  77 St. Clair Ave E     2126.000 1080000.000      5049.000   
3     341    40 Pleasant Blvd     2387.000 1930000.000      5662.000   
4     208    40 Pleasant Blvd     1337.000 2350000.000      7929.000   
..    ...                 ...          ...         ...           ...   
75   1203     125 Lawton Blvd      911.000  785000.000      3125.000   
76      3      95 Lawton Blvd     2383.000 1499

In [32]:
# Function to calculate appreciation in the stock market
def calc_etf(rent,years=20):
    rent_tracker = [rent]
    for c in list(range(0,years)):
        rent2 = rent_tracker[-1] + (rent_tracker[-1]*0.0624)
        rent_tracker.append(rent2)
    byear = rent_tracker

    return byear[-1]

In [35]:
# Function to calculate rent paid
def calc_rent_increase(rent,years=20):
    rent_tracker = [rent]
    for c in list(range(0,years)):
        rent2 = rent_tracker[-1] + (rent_tracker[-1]*0.015)
        rent_tracker.append(rent2)

    byear = [x*12 for x in rent_tracker]
    return sum(byear)

In [46]:
# Now we can compare to performance in the market (ETF or 5% GIC)
bcsv2['mpay_re'] = bcsv2['mpay']-bcsv2['Maintenance']-bcsv2['Property Tax']/12
rental_cost = bcsv2['mpay_re'].apply(calc_rent_increase)
bcsv2['dp_etf'] = bcsv2['down_pay'].apply(calc_etf) - rental_cost
bcsv2['diff'] = bcsv2['total_buy2']-bcsv2['dp_etf']
pd.options.display.max_columns = None
print('CONDOS THAT DID WORSE THAN STOCK MARKET RETURNS (20 YEARS)..............')
print(bcsv2[bcsv2['diff'] < 0][['Address','Cost','Bedrooms','total_buy2','appreciation','down_pay','dp_etf','mpay']])

gic_better = bcsv2[bcsv2['diff'] < 0][['Address','Cost','Bedrooms','total_buy2','appreciation','down_pay','dp_etf','mpay']]
gic_better_p = round(len(gic_better) / len(bcsv2) *100,2)
print('Percent of condo investments with 20 percent down that are worse than ETF: %s percent'%(gic_better_p))

print('Top 5% Performing Condos...............................................')
p95 = np.percentile(bcsv2['total_buy2'],95)
print(bcsv2[bcsv2['total_buy2'] >= p95])

print('Botton 5% Performing Condos...............................................')
p5 = np.percentile(bcsv2['total_buy2'],5)
print(bcsv2[bcsv2['total_buy2'] <= p5])


CONDOS THAT DID WORSE THAN STOCK MARKET RETURNS (20 YEARS)..............
          Address       Cost  Bedrooms  total_buy2  appreciation   down_pay  \
17  587 Avenue Rd 925000.000     2.000 -596722.700    242319.000 185000.000   

        dp_etf     mpay  
17 -552588.321 7399.000  
Percent of condo investments with 20 percent down that are worse than ETF: 1.3 percent
Top 5% Performing Condos...............................................
   Number               Address  Maintenance        Cost  Property Tax  \
5     617      40 Pleasant Blvd     1735.000 3400000.000     10307.000   
8     520        70 Delisle Ave      758.000 1265000.000      3931.000   
12    503  64 St Clair Ave West     2218.000 4100000.000      8903.000   
22   1002       606 Avenue Road     1930.000 2750000.000      7295.000   

    Bedrooms  Bath  Sq Foot  Last Sold   Price Last Sold latitude longitude  \
5      2.000 3.000 1800.000    2018.000      3300000.000   43.688   -79.392   
8      2.000 2.000  800.000 